In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
path_to_csv = os.path.abspath('cache/import.csv')

In [3]:
current_data = pd.read_csv('cache/current.csv')

In [6]:
preds = pd.read_csv('cache/batch_pred.csv', index_col=0)

In [9]:
preds.iloc[1942:]

,Categorical,Date,Url,Pred
1942,economic,30/05/2024,https://vtv.vn/kinh-te/gia-dau-the-gioi-giam-2...,Giá dầu thế giới giảm mạnh và lượng tiền gửi v...
1943,economic,30/05/2024,https://vtv.vn/kinh-te/loat-chinh-sach-kinh-te...,Nghị định 45/2024/NĐ-CP quy định về phương phá...
1944,economic,30/05/2024,https://vtv.vn/kinh-te/cam-do-co-phai-tin-dung...,"Theo nhận định của các chuyên gia, việc kinh d..."
1945,economic,30/05/2024,https://vtv.vn/kinh-te/3-luat-moi-ve-bat-dong-...,"Chính phủ vừa có văn bản gửi Quốc hội xem xét,..."
1946,economic,30/05/2024,https://vtv.vn/kinh-te/elon-musk-gianh-lai-ngo...,Công ty khởi nghiệp trí tuệ nhân tạo Elon Musk...
...,...,...,...,...
2186,economic,21/05/2024,https://vtv.vn/kinh-te/co-hoi-ve-huu-som-cho-n...,3.2.1. Đề xuất tăng tuổi nghỉ hưu từ năm 2023 ...
2187,economic,21/05/2024,https://vtv.vn/kinh-te/han-quoc-ky-vong-xuat-k...,Xuất khẩu mì ăn liền của Hàn Quốc vượt 1 tỷ US...
2188,economic,21/05/2024,https://vtv.vn/kinh-te/khoanh-28380-ty-dong-ti...,"Theo Bộ Tài chính, việc xử lý nợ theo Nghị quy..."
2189,economic,21/05/2024,https://vtv.vn/kinh-te/huong-tin-dung-vao-san-...,"Theo thống kê, từ đầu năm đến nay, nhiều ngân ..."


In [10]:
import psycopg2 

connection = psycopg2.connect(database='postgres',
                              user='postgres',
                              password='123456a@A',
                              host='localhost',
                              port='5432',
                              options="-c search_path=pbl7,public")

In [30]:
def get_categories():
    cur = connection.cursor()
    cur.execute('select * from "PBL7".category')
    categories = cur.fetchall()
    
    return categories

In [13]:
categories = get_categories()

In [14]:
def getCategoryId(name):
    for category in categories:
        if category[1] == name: return category[0]
    
    return ''

def convert_category(category: str):
    if category == 'economic': 
        return getCategoryId('Kinh tế')
    if category == 'social':
        return getCategoryId('Xã hội')
    
    return getCategoryId(category)

def convert_date(date: str):
    [dd, mm, yyyy] = date.split('/')
    return '-'.join([yyyy, mm, dd])

def apply_convert(_df: pd.DataFrame):
    df = _df.copy()
    df['Categorical'] = df['Categorical'].apply(convert_category)
    df['Date'] = df['Date'].apply(convert_date)
    return df

In [22]:
import_data = apply_convert(preds.iloc[1942:])
import_data.reset_index(inplace=True)

In [23]:
import_data.index += current_data.index.stop + 1

In [24]:
import_data

,index,Categorical,Date,Url,Pred
3451,1942,clwrok1q600ml7laqn3i1gqci,2024-05-30,https://vtv.vn/kinh-te/gia-dau-the-gioi-giam-2...,Giá dầu thế giới giảm mạnh và lượng tiền gửi v...
3452,1943,clwrok1q600ml7laqn3i1gqci,2024-05-30,https://vtv.vn/kinh-te/loat-chinh-sach-kinh-te...,Nghị định 45/2024/NĐ-CP quy định về phương phá...
3453,1944,clwrok1q600ml7laqn3i1gqci,2024-05-30,https://vtv.vn/kinh-te/cam-do-co-phai-tin-dung...,"Theo nhận định của các chuyên gia, việc kinh d..."
3454,1945,clwrok1q600ml7laqn3i1gqci,2024-05-30,https://vtv.vn/kinh-te/3-luat-moi-ve-bat-dong-...,"Chính phủ vừa có văn bản gửi Quốc hội xem xét,..."
3455,1946,clwrok1q600ml7laqn3i1gqci,2024-05-30,https://vtv.vn/kinh-te/elon-musk-gianh-lai-ngo...,Công ty khởi nghiệp trí tuệ nhân tạo Elon Musk...
...,...,...,...,...,...
3695,2186,clwrok1q600ml7laqn3i1gqci,2024-05-21,https://vtv.vn/kinh-te/co-hoi-ve-huu-som-cho-n...,3.2.1. Đề xuất tăng tuổi nghỉ hưu từ năm 2023 ...
3696,2187,clwrok1q600ml7laqn3i1gqci,2024-05-21,https://vtv.vn/kinh-te/han-quoc-ky-vong-xuat-k...,Xuất khẩu mì ăn liền của Hàn Quốc vượt 1 tỷ US...
3697,2188,clwrok1q600ml7laqn3i1gqci,2024-05-21,https://vtv.vn/kinh-te/khoanh-28380-ty-dong-ti...,"Theo Bộ Tài chính, việc xử lý nợ theo Nghị quy..."
3698,2189,clwrok1q600ml7laqn3i1gqci,2024-05-21,https://vtv.vn/kinh-te/huong-tin-dung-vao-san-...,"Theo thống kê, từ đầu năm đến nay, nhiều ngân ..."


In [26]:
path_to_csv

'd:\\code\\pbl7\\src\\import.csv'

In [27]:
cur = connection.cursor()

cur.execute('''
            COPY "PBL7".news("id", "categoryId", "Date", "url", "summary") 
            FROM '{}'
            DELIMITER ','
            CSV HEADER;
            '''.format(path_to_csv))

In [28]:
connection.commit()
cur.close()
preds.to_csv('cache/current.csv', index=False, header=False, mode='a')

In [29]:
connection.close()